# Day 2: Supply I

We talked today about how electricity markets work.

We will learn today how to build a simple model of an electricity market using **JuMP**.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.


We first load relevant libraries.

Compared to day 1, we will be adding the libraries `JuMP` and the non-linear solver `Ipopt`. We will also be using the clustering library `Clustering` and the package `Random` to control the randomness in the machine learning algorithm.

**Note:** I often prefer to use commercial solvers (Gurobi or CPLEX), which are available under an academic license. I use solvers that are readily available here without a license for simplicity and to ensure that everyone can access the code.


In [13]:
#using Pkg
#Pkg.add([ "StatsPlots","StatsBase","Clustering","Random","JuMP","Ipopt","Printf"])

using DataFrames
using CSV
using Plots
using StatsPlots
using Statistics, StatsBase
using Clustering
using Random
using JuMP
using Ipopt
using Printf
	


We load the data using the CSV syntax (`CSV.read`) into a data frame called `df`.  Here we need to do some cleaning of the variables, rescaling and dropping missing entries.

Having a clean dataset will be very helpful for the clustering algorithm, which requires complete rows of data.


In [3]:
# We read the data and clean it up a bit
df = CSV.read("data_jaere.csv", DataFrame)
df = sort(df,["year","month","day","hour"])
df = dropmissing(df)
df.nuclear = df.nuclear/1000.0
df.hydro = df.hydro/1000.0
df.imports = df.imports/1000.0
df.q_commercial = df.q_commercial/1000.0
df.q_industrial = df.q_industrial/1000.0
df.q_residential = df.q_residential/1000.0
df.hydronuc = df.nuclear + df.hydro 
df = select(df,Not(["nuclear","hydro"]))
first(df, 5)

,year,month,day,hour,price,imports,q_commercial,q_industrial,q_residential
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,2011,1,2,1,29.5397,4.502,8.38001,2.05659,10.6404
2,2011,1,2,2,27.9688,4.363,8.34789,2.06558,9.80354
3,2011,1,2,3,26.5258,4.089,8.54809,2.11851,9.5554
4,2011,1,2,4,25.5872,3.783,8.56002,2.13467,9.31031
5,2011,1,2,5,25.9229,3.969,8.61251,2.17499,9.4285


In [4]:
 describe(df)
# nrow(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,year,2013.0,2011,2013.0,2015,0,Int64
2,month,6.5478,1,7.0,12,0,Int64
3,day,15.7486,1,16.0,31,0,Int64
4,hour,12.5064,1,13.0,24,0,Int64
5,price,35.5433,-13.9395,34.5713,172.352,0,Float64
6,imports,7.41422,1.571,7.446,11.674,0,Float64
7,q_commercial,12.1097,6.91611,11.458,26.0133,0,Float64
8,q_industrial,3.91344,1.95289,3.65441,8.53145,0,Float64
9,q_residential,10.5988,3.87066,9.8203,24.9831,0,Float64


## Clustering

When modeling electricity markets, oftentimes the size of the problem can make the solver slow.

Here we will be using a clustering algorithm to come up with a (much) smaller synthetic dataset that we will use for the purposes of our main analysis.

**Note:** We ignore the time variables when we cluster. In our case, we'll transform a dataset of 43408 hours to 100 representatives hours. That is enough if you assume no correlation between hours.


In [5]:
# define number of clusters
n = 100
# Clustering algorithms work in rows, so we need to transpose our data
X = transpose(Array(select(df,Between(:price,:hydronuc))));

# We scale variables to improve kmeans performance. For that, we take the mean and std of each row (dim=2) and you repeat it by the number of columns (rows in df)
Xs = (X.- repeat(mean(X,dims=2),1,nrow(df)))./repeat(std(X,dims=2),1,nrow(df)); 

#we set seed because kmeans picks random samples to generate clusters
Random.seed!(2020)
R = kmeans(Xs, n);

# Get the cluster centers rescaling again. These centers will be the new observations
M = R.centers .* repeat(std(X,dims=2),1,n) .+ repeat(mean(X,dims=2),1,n);  

# Assignments of each cluster
A = assignments(R)


43408-element Vector{Int64}:
  23
  23
  23
  23
  23
  23
  23
  39
  39
  74
   ⋮
  46
  52
 100
 100
 100
 100
  52
  52
  52

In [ ]:
dfclust = DataFrame(transpose(M),
	["price", "imports", "q_commercial", "q_industrial", "q_residential", 
			"wind_cap", "solar_cap", "hydronuc"]);
    # the weights is defined by the number of old observations assigned to each cluster          
	dfclust.weights = counts(R);
	first(dfclust, 5)
	

We can compare the distribution of outcomes between the original dataset and the new dataset.

Here is an example with prices. The two distributions are very similar.


In [ ]:
histogram(df.price, fillalpha=.2, nbins=20, label="Data")
	histogram!(dfclust.price,weights=dfclust.weights, 
	fillalpha=.2, nbins=20, 
		label="Clusters")

It is also relatively well matched for the case for solar, although it is harder there.


In [ ]:
histogram(df.solar_cap, fillalpha=.2, nbins=20, label="Original")
	histogram!(dfclust.solar_cap, weights=dfclust.weights, fillalpha=.2, nbins=20, label="Clusters")

### Saving the output

It is useful to save the clustered data so that we can use it directly.


In [ ]:
CSV.write("data_jaere_clustered.csv", dfclust)


## Building the model

Now that we have clustered our data, we will build our model with the data that we have. 

The model that we will build today is a simplification from the original paper.

In the original paper, the model needed to solve for:
1. Endogenous retail prices (in a demand model, iterated to find equilibrium)
2. Endogenous investment (in same supply model, with more equations)

Here we will be simply building a simple model of market clearing.


Before building the model, we define some model parameters related to:

* Number and costs of different technologies (loaded from a small dataset)

* Elasticity of demand and imports


In [ ]:
tech = CSV.read("data_technology_simple.csv", DataFrame)


Here we assume zero marginal costs for renewable energy. `capUB` defines the maximum capacity of each power plant. For the existing ones, it defines the maximum production. For renewables, they will always produce at maximum capacity (precisely because of zero marginal costs) but production will depend on the capacity factors defined before.

To calibrate demand, one can use different strategies. Here we compute the slope for the demand curve that is consistent with the assumed elasticity of demand. 

Notice that this is a local elasticity approximation, but it has the advantage of being a linear demand curve, which is very attractive for the purposes of linear programming.

The demand is: $ q = a - b \ p $

So the elasticity becomes: $elas =  b  \frac{p}{q} $, which we set equal to an assumed parameter.

Once we have $b$, we can back out $a$. 

An analogous procedure is done for imports, but in this case, $ qm = am + bm \ p $


In [ ]:
dfclust.weights = dfclust.weights / sum(dfclust.weights);
	
	# Here only one demand type to make it easier
	dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;
	
    # Calibrate demand based on elasticities (using 0.1 here as only one final demand)
	elas = [.1, .2, .5, .3];
	dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
	dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

	# Calibrate imports (using elas 0.3)
    dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
    dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

In [ ]:
describe(dfclust)

### Non-linear solver

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$ \max \ CS - Costs $

$ \text{s.t.} \ \text{operational constraints, market clearing}. $


In [ ]:
function clear_market_max(data::DataFrame, tech::DataFrame; 
    wind_gw = 5.0, solar_gw = 2.0)

# We declare a model
model = Model(
    optimizer_with_attributes(
        Ipopt.Optimizer)
    );

# Set useful indexes
I = nrow(tech);  # number of techs
T = nrow(data);  # number of periods
S = 1;  # we will only be using one sector to keep things simple

# Variables to solve for
@variable(model, price[1:T]);
@variable(model, demand[1:T]);
@variable(model, imports[1:T]);
@variable(model, quantity[1:T, 1:I] >= 0);
@variable(model, Q[1:T] >= 0);

# Maximize welfare including imports costs
@NLobjective(model, Max, sum(data.weights[t] * (
            (data.a[t] - demand[t]) * demand[t] / data.b[t] 
        + demand[t]^2/(2*data.b[t])
    - sum(tech.c[i] * quantity[t,i] 
                + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
    - (imports[t] - data.am[t])^2/(2 * data.bm[t])) for t=1:T));

# Market clearing
@constraint(model, [t=1:T], 
    demand[t] == data.a[t] - data.b[t] * price[t]);
@constraint(model, [t=1:T], 
    imports[t] == data.am[t] + data.bm[t] * price[t]);
@constraint(model, [t=1:T], 
    demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);
@constraint(model, [t=1:T], 
    Q[t] == sum(quantity[t,i] for i=1:I) );

# Constraints on output
@constraint(model, [t=1:T], 
    quantity[t,1] <= data.hydronuc[t]);	
@constraint(model, [t=1:T,i=2:4], 
    quantity[t,i] <= tech[i,"capUB"]);
@constraint(model, [t=1:T], 
    quantity[t,5] <= wind_gw * data.wind_cap[t]);
@constraint(model, [t=1:T], 
    quantity[t,6] <= solar_gw * data.solar_cap[t]);

# Solve model
optimize!(model);

status = @sprintf("%s", JuMP.termination_status(model));

if (status=="LOCALLY_SOLVED")
    p = JuMP.value.(price);
    avg_price = sum(p[t] * data.weights[t] for t=1:T);
    q = JuMP.value.(quantity);
    Q=JuMP.value.(Q);
    imp = JuMP.value.(imports);
    d = JuMP.value.(demand);
    cost = sum(data.weights[t] * (sum(tech.c[i] * q[t,i] 
            + tech.c2[i] * q[t,i]^2 / 2 for i=1:I) 
            + (imp[t] - data.am[t])^2/(2 * data.bm[t])) for t=1:T);
    results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
        "avg_price" => avg_price,
        "price" => p,
        "quantity" => q,
        "Total_quantity" => Q
        "imports" => imp,
        "demand" => d,
        "cost" => cost);
    return results
else
    results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
    return results
end

end

In [ ]:
results = clear_market_max(dfclust, tech)


In [ ]:
results["avg_price"]

We can study the effects of an increase in renewable capacity

In [ ]:
results = clear_market_max(dfclust, tech, wind_gw = 7.0, solar_gw = 3.0)